<a href="https://colab.research.google.com/github/Chediak/common-master-ai/blob/main/redhat_prodesan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install pdfplumber sentence-transformers faiss-cpu

In [8]:
import pdfplumber
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import json

# Step 1: Extract text from a PDF
def extract_text_from_pdf(pdf_path):
    with pdfplumber.open(pdf_path) as pdf:
        pages = [page.extract_text() for page in pdf.pages]
    return pages

# Step 2: Preprocess and split into individual news items
def preprocess_and_split_text(pages):
    news_items = [item.strip() for page in pages for item in page.split("\n\n") if item.strip()]
    return news_items

# Step 3: Generate embeddings using SentenceTransformer
def generate_embeddings(news_items, model_name='all-MiniLM-L6-v2'):
    model = SentenceTransformer(model_name)
    embeddings = model.encode(news_items)
    return embeddings, model

# Step 4: Store embeddings in a FAISS vector store
def create_faiss_index(embeddings):
    dimension = embeddings.shape[1]
    index = faiss.IndexFlatL2(dimension)
    index.add(embeddings)
    return index

# Step 5: Add metadata for later retrieval
def add_metadata(news_items):
    metadata = [{"id": i, "content": news_items[i]} for i in range(len(news_items))]
    return metadata

# Step 6: Query the vector store
def search_similar_news(query, model, index, metadata, top_k=5):
    query_embedding = model.encode([query])
    distances, indices = index.search(query_embedding, top_k)
    results = [
        {"content": metadata[idx]["content"], "distance": float(dist)}
        for dist, idx in zip(distances[0], indices[0])
    ]
    return results

# Full Pipeline Test
def main():
    # File path to the test PDF
    pdf_path = "/content/DOM-1947.pdf"  # Replace with your PDF path

    # Extract text
    print("Extracting text from PDF...")
    pages = extract_text_from_pdf(pdf_path)

    # Preprocess and split
    print("Preprocessing and splitting text...")
    news_items = preprocess_and_split_text(pages)

    # Generate embeddings
    print("Generating embeddings...")
    embeddings, model = generate_embeddings(news_items)

    # Create FAISS index
    print("Creating FAISS index...")
    index = create_faiss_index(np.array(embeddings))

    # Add metadata
    print("Adding metadata...")
    metadata = add_metadata(news_items)

    # Test a query
    query = "example query about a topic"  # Replace with your query
    print("Searching for similar news...")
    results = search_similar_news(query, model, index, metadata, top_k=5)

    # Print results
    print("Results:")
    print(json.dumps(results, indent=2))

if __name__ == "__main__":
    main()

Extracting text from PDF...
Preprocessing and splitting text...
Generating embeddings...
Creating FAISS index...
Adding metadata...
Searching for similar news...
Results:
[
  {
    "content": "Quarta-feira, 13 de novembro de 2024 03 Ano X \u2022 N\u00ba 1.947 \u2022 Prefeitura Municipal de Guara\u00ed/TO\nSECRETARIA MUNICIPAL DE SA\u00daDE Instala\u00e7\u00e3o, desinstala\u00e7\u00e3o, manuten\u00e7\u00e3o,\nlimpeza, reparo em geladeira,\n100 60,00 6.000,00\nHORA bebedouro, refrigerador, frigobar,\nfreezer, filtro e purificador de \u00e1gua\nEXTRATO DO PRIMEIRO TERMO ADITIVO 01 Valor global estimado para aquisi\u00e7\u00e3o\nde pe\u00e7as e/ou componentes originais\nem geladeira, bebedouro, refrigerador, 6.000,00\nCONTRATO N.\u00ba 073/2023\nfrigobar, freezer, filtro e purificador\nProcesso: 3243/2023 de \u00e1gua\nPreg\u00e3o Eletr\u00f4nico: 028/2023\nDESCONTO NO VALOR DAS PE\u00c7AS - 40 %\n\u00d3rg\u00e3o: Fundo Municipal da Sa\u00fade de Guara\u00ed - TO.\nContratada: ARAUJO E REP

In [9]:
import pdfplumber
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import json

# Step 1: Extract text from a PDF
def extract_text_from_pdf(pdf_path):
    with pdfplumber.open(pdf_path) as pdf:
        pages = [page.extract_text() for page in pdf.pages]
    return pages

# Step 2: Preprocess and split into individual news items
def preprocess_and_split_text(pages):
    news_items = [item.strip() for page in pages for item in page.split("\n\n") if item.strip()]
    return news_items

# Step 3: Generate embeddings using SentenceTransformer
def generate_embeddings(news_items, model_name='all-MiniLM-L6-v2'):
    model = SentenceTransformer(model_name)
    embeddings = model.encode(news_items)
    return embeddings, model

# Step 4: Store embeddings in a FAISS vector store
def create_faiss_index(embeddings):
    dimension = embeddings.shape[1]
    index = faiss.IndexFlatL2(dimension)
    index.add(embeddings)
    return index

# Step 5: Add metadata for later retrieval
def add_metadata(news_items):
    metadata = [{"id": i, "content": news_items[i], "length": len(news_items[i])} for i in range(len(news_items))]
    return metadata

# Step 6: Query the vector store
def search_similar_news(query, model, index, metadata, top_k=5):
    query_embedding = model.encode([query])
    distances, indices = index.search(query_embedding, top_k)
    results = [
        {"content": metadata[idx]["content"], "distance": float(dist), "length": metadata[idx]["length"]}
        for dist, idx in zip(distances[0], indices[0])
    ]
    return results

# Step 7: Reorganize for Optimization of Space
def optimize_layout(metadata, top_k=5):
    # Sort news items by length (longer news first)
    sorted_news = sorted(metadata, key=lambda x: x["length"], reverse=True)

    # Create layout with strategic placement of advertisements
    layout = []
    for i, item in enumerate(sorted_news):
        layout.append({"type": "news", "content": item["content"], "page": i // top_k + 1, "position": i % top_k})
        # Add an advertisement after every 3 news items
        if (i + 1) % 3 == 0:
            layout.append({"type": "advertisement", "content": "Ad Placeholder", "page": (i + 1) // top_k + 1, "position": "bottom"})

    return layout

# Full Pipeline Test
def main():
    # File path to the test PDF
    pdf_path = "/content/DOM-1947.pdf"  # Replace with your PDF path

    # Extract text
    print("Extracting text from PDF...")
    pages = extract_text_from_pdf(pdf_path)

    # Preprocess and split
    print("Preprocessing and splitting text...")
    news_items = preprocess_and_split_text(pages)

    # Generate embeddings
    print("Generating embeddings...")
    embeddings, model = generate_embeddings(news_items)

    # Create FAISS index
    print("Creating FAISS index...")
    index = create_faiss_index(np.array(embeddings))

    # Add metadata
    print("Adding metadata...")
    metadata = add_metadata(news_items)

    # Test a query
    query = "example query about a topic"  # Replace with your query
    print("Searching for similar news...")
    results = search_similar_news(query, model, index, metadata, top_k=5)

    # Print search results
    print("Search Results:")
    print(json.dumps(results, indent=2))

    # Optimize layout
    print("Optimizing layout...")
    layout = optimize_layout(metadata)

    # Print optimized layout
    print("Layout:")
    print(json.dumps(layout, indent=2))

if __name__ == "__main__":
    main()


Extracting text from PDF...
Preprocessing and splitting text...
Generating embeddings...
Creating FAISS index...
Adding metadata...
Searching for similar news...
Search Results:
[
  {
    "content": "Quarta-feira, 13 de novembro de 2024 03 Ano X \u2022 N\u00ba 1.947 \u2022 Prefeitura Municipal de Guara\u00ed/TO\nSECRETARIA MUNICIPAL DE SA\u00daDE Instala\u00e7\u00e3o, desinstala\u00e7\u00e3o, manuten\u00e7\u00e3o,\nlimpeza, reparo em geladeira,\n100 60,00 6.000,00\nHORA bebedouro, refrigerador, frigobar,\nfreezer, filtro e purificador de \u00e1gua\nEXTRATO DO PRIMEIRO TERMO ADITIVO 01 Valor global estimado para aquisi\u00e7\u00e3o\nde pe\u00e7as e/ou componentes originais\nem geladeira, bebedouro, refrigerador, 6.000,00\nCONTRATO N.\u00ba 073/2023\nfrigobar, freezer, filtro e purificador\nProcesso: 3243/2023 de \u00e1gua\nPreg\u00e3o Eletr\u00f4nico: 028/2023\nDESCONTO NO VALOR DAS PE\u00c7AS - 40 %\n\u00d3rg\u00e3o: Fundo Municipal da Sa\u00fade de Guara\u00ed - TO.\nContratada: ARAUJ

In [10]:
import spacy
from datetime import datetime
import re

# Load spaCy model for NER
nlp = spacy.load("en_core_web_sm")  # Use a language-specific model, e.g., 'pt_core_news_sm' for Portuguese

# Extract entities from text
def extract_entities(news_items):
    extracted_data = []
    for idx, news in enumerate(news_items):
        doc = nlp(news)
        entities = {"id": idx, "content": news, "dates": [], "names": [], "organizations": []}

        for ent in doc.ents:
            if ent.label_ in ["DATE"]:
                entities["dates"].append(ent.text)
            elif ent.label_ in ["PERSON"]:
                entities["names"].append(ent.text)
            elif ent.label_ in ["ORG"]:
                entities["organizations"].append(ent.text)

        # Regex for CNPJ or specific patterns
        cnpj_matches = re.findall(r"\d{2}\.\d{3}\.\d{3}/\d{4}-\d{2}", news)
        entities["cnpjs"] = cnpj_matches
        extracted_data.append(entities)
    return extracted_data

# Create a structured index for fast retrieval
def create_structured_index(entities):
    index = {}
    for item in entities:
        for date in item["dates"]:
            date_key = datetime.strptime(date, "%d/%m/%Y").date() if re.match(r"\d{2}/\d{2}/\d{4}", date) else date
            index.setdefault("dates", {}).setdefault(date_key, []).append(item)
        for name in item["names"]:
            index.setdefault("names", {}).setdefault(name, []).append(item)
        for org in item["organizations"]:
            index.setdefault("organizations", {}).setdefault(org, []).append(item)
        for cnpj in item["cnpjs"]:
            index.setdefault("cnpjs", {}).setdefault(cnpj, []).append(item)
    return index

# Query structured index
def query_index(index, query_type, query_value):
    if query_type not in index:
        return []
    return index.get(query_type, {}).get(query_value, [])

# Example integration
def main_with_structured_search():
    # Example news items
    news_items = [
        "Em 13/11/2024, a Prefeitura de Guaraí contratou ARAUJO E REPLANDE LTDA para manutenção de ar condicionado.",
        "A empresa PONTUAL REFRIGERAÇÃO assinou contrato em 12/11/2024 com a Prefeitura Municipal de Guaraí.",
    ]

    # Extract entities
    print("Extracting entities...")
    entities = extract_entities(news_items)

    # Create structured index
    print("Creating structured index...")
    index = create_structured_index(entities)

    # Query examples
    print("Querying for '13/11/2024' in dates...")
    results = query_index(index, "dates", datetime.strptime("13/11/2024", "%d/%m/%Y").date())
    print(results)

    print("Querying for 'ARAUJO E REPLANDE LTDA' in organizations...")
    results = query_index(index, "organizations", "ARAUJO E REPLANDE LTDA")
    print(results)

if __name__ == "__main__":
    main_with_structured_search()


Extracting entities...
Creating structured index...
Querying for '13/11/2024' in dates...
[{'id': 0, 'content': 'Em 13/11/2024, a Prefeitura de Guaraí contratou ARAUJO E REPLANDE LTDA para manutenção de ar condicionado.', 'dates': ['13/11/2024'], 'names': ['Prefeitura de Guaraí'], 'organizations': ['REPLANDE LTDA', 'manutenção de'], 'cnpjs': []}]
Querying for 'ARAUJO E REPLANDE LTDA' in organizations...
[]
